In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import joblib
import optuna

In [3]:
print("="*60)
print("ENHANCED TRAINING WITH FEATURE ENGINEERING")
print("="*60)
# Load data
df = pd.read_csv("../backend/data/preprocessed_data.csv")
print(f"\nOriginal shape: {df.shape}")
print(f"Original columns: {df.columns.tolist()}")
# Remove Year
if 'Year' in df.columns:
    df = df.drop('Year', axis=1)
    print("✅ Year removed")

ENHANCED TRAINING WITH FEATURE ENGINEERING

Original shape: (266733, 8)
Original columns: ['State', 'Crop', 'Year', 'Season', 'Yield', 'Annual_Rainfall', 'Fertilizer', 'Pesticide']
✅ Year removed


In [5]:
# ========================================
# FEATURE ENGINEERING (KEY IMPROVEMENT!)
# ========================================
print("\n🔧 Creating engineered features...")
# 1. Interaction features
df['Rainfall_per_Fertilizer'] = df['Annual_Rainfall'] / (df['Fertilizer'] + 1)
df['Fertilizer_Pesticide_Ratio'] = df['Fertilizer'] / (df['Pesticide'] + 0.01)
df['Total_Input'] = df['Fertilizer'] + df['Pesticide']
# 2. Polynomial features for key numeric columns
df['Rainfall_Squared'] = df['Annual_Rainfall'] ** 2
df['Fertilizer_Squared'] = df['Fertilizer'] ** 2
# 3. Log transformations (helps with skewed distributions)
df['Log_Rainfall'] = np.log1p(df['Annual_Rainfall'])
df['Log_Fertilizer'] = np.log1p(df['Fertilizer'])
print(f"✅ Added {len(df.columns) - 7} engineered features")
print(f"New feature count: {len(df.columns) - 1} (excluding Yield)")



🔧 Creating engineered features...
✅ Added 7 engineered features
New feature count: 13 (excluding Yield)


In [7]:
# ========================================
# Encoding
# ========================================
df_encoded = df.copy()
encoders = {}
categorical_cols = df_encoded.select_dtypes(include=['object']).columns.tolist()
for col in categorical_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    encoders[col] = le
print(f"\n✅ Encoded {len(categorical_cols)} categorical columns")


✅ Encoded 3 categorical columns


In [9]:
# Split features and target
X = df_encoded.drop('Yield', axis=1)
y = df_encoded['Yield']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"\nTrain size: {len(X_train)}, Test size: {len(X_test)}")


Train size: 213386, Test size: 53347


In [11]:
# ========================================
# Prepare categorical versions for LightGBM
# ========================================
X_train_lgb = X_train.copy()
X_test_lgb = X_test.copy()
for col in categorical_cols:
    if col in X_train_lgb.columns:
        X_train_lgb[col] = X_train_lgb[col].astype('category')
        X_test_lgb[col] = X_test_lgb[col].astype('category')

In [13]:
# ========================================
# Train Individual Models
# ========================================
print("\n" + "="*60)
print("TRAINING INDIVIDUAL MODELS")
print("="*60)
# Random Forest
print("\n1. Training Random Forest...")
rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_r2 = r2_score(y_test, rf_pred)
rf_mae = mean_absolute_error(y_test, rf_pred)
print(f"   R²={rf_r2:.4f}, MAE={rf_mae:.4f}")
# XGBoost
print("\n2. Training XGBoost...")
xgb = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=8,
    random_state=42,
    n_jobs=-1
)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
xgb_r2 = r2_score(y_test, xgb_pred)
xgb_mae = mean_absolute_error(y_test, xgb_pred)
print(f"   R²={xgb_r2:.4f}, MAE={xgb_mae:.4f}")


TRAINING INDIVIDUAL MODELS

1. Training Random Forest...
   R²=0.8802, MAE=22.0552

2. Training XGBoost...
   R²=0.8791, MAE=22.3513


In [15]:
# LightGBM with Optuna
print("\n3. Training LightGBM with Optuna (50 trials)...")
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 31, 200),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'random_state': 42
    }
    
    model = LGBMRegressor(**params)
    scores = cross_val_score(
        model, X_train_lgb, y_train,
        cv=3, scoring='r2', n_jobs=-1,
        fit_params={'categorical_feature': 'auto'}
    )
    return scores.mean()
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50, show_progress_bar=True)
print(f"\n✅ Best LightGBM params found!")
best_params = study.best_params
best_lgb = LGBMRegressor(**best_params)
best_lgb.fit(X_train_lgb, y_train, categorical_feature='auto')
lgb_pred = best_lgb.predict(X_test_lgb)
lgb_r2 = r2_score(y_test, lgb_pred)
lgb_mae = mean_absolute_error(y_test, lgb_pred)
print(f"   R²={lgb_r2:.4f}, MAE={lgb_mae:.4f}")

[I 2025-11-30 12:28:05,780] A new study created in memory with name: no-name-7e2f3c86-cb3b-40a9-9483-39f2474f438d



3. Training LightGBM with Optuna (50 trials)...


  0%|          | 0/50 [00:00<?, ?it/s]

C:\Users\Tejas\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(


[I 2025-11-30 12:28:21,513] Trial 0 finished with value: 0.8682687230610423 and parameters: {'n_estimators': 562, 'learning_rate': 0.09556428757689246, 'num_leaves': 155, 'max_depth': 14, 'min_child_samples': 24, 'subsample': 0.7467983561008608, 'colsample_bytree': 0.7174250836504598, 'reg_alpha': 0.8661761457749352, 'reg_lambda': 0.6011150117432088}. Best is trial 0 with value: 0.8682687230610423.
[I 2025-11-30 12:28:40,492] Trial 1 finished with value: 0.86947737727731 and parameters: {'n_estimators': 796, 'learning_rate': 0.011852604486622221, 'num_leaves': 195, 'max_depth': 18, 'min_child_samples': 29, 'subsample': 0.7545474901621302, 'colsample_bytree': 0.7550213529560301, 'reg_alpha': 0.3042422429595377, 'reg_lambda': 0.5247564316322378}. Best is trial 1 with value: 0.86947737727731.
[I 2025-11-30 12:28:49,366] Trial 2 finished with value: 0.8700091865708101 and parameters: {'n_estimators': 602, 'learning_rate': 0.036210622617823776, 'num_leaves': 135, 'max_depth': 7, 'min_child_

In [17]:
# ========================================
# ENSEMBLE MODEL (KEY IMPROVEMENT!)
# ========================================
print("\n" + "="*60)
print("CREATING ENSEMBLE MODEL")
print("="*60)
# Weighted ensemble based on individual performance
total_r2 = rf_r2 + xgb_r2 + lgb_r2
rf_weight = rf_r2 / total_r2
xgb_weight = xgb_r2 / total_r2
lgb_weight = lgb_r2 / total_r2
print(f"\nWeights: RF={rf_weight:.3f}, XGB={xgb_weight:.3f}, LGB={lgb_weight:.3f}")
# Simple weighted average
ensemble_pred = (rf_pred * rf_weight + xgb_pred * xgb_weight + lgb_pred * lgb_weight)
ensemble_r2 = r2_score(y_test, ensemble_pred)
ensemble_mae = mean_absolute_error(y_test, ensemble_pred)
print(f"\n🏆 Ensemble Performance:")
print(f"   R²={ensemble_r2:.4f}, MAE={ensemble_mae:.4f}")



CREATING ENSEMBLE MODEL

Weights: RF=0.333, XGB=0.333, LGB=0.334

🏆 Ensemble Performance:
   R²=0.8817, MAE=22.3444


In [19]:
# ========================================
# Final Comparison
# ========================================
print("\n" + "="*60)
print("FINAL MODEL COMPARISON")
print("="*60)
results = {
    'Random Forest': (rf_r2, rf_mae),
    'XGBoost': (xgb_r2, xgb_mae),
    'LightGBM (Tuned)': (lgb_r2, lgb_mae),
    'Ensemble': (ensemble_r2, ensemble_mae)
}
for name, (r2, mae) in results.items():
    print(f"{name:20s}: R²={r2:.4f}, MAE={mae:.4f}")



FINAL MODEL COMPARISON
Random Forest       : R²=0.8802, MAE=22.0552
XGBoost             : R²=0.8791, MAE=22.3513
LightGBM (Tuned)    : R²=0.8825, MAE=23.4781
Ensemble            : R²=0.8817, MAE=22.3444


In [21]:
# Select best model
best_model_name = max(results, key=lambda x: results[x][0])
print(f"\n🏆 Best model: {best_model_name} (R²={results[best_model_name][0]:.4f})")


🏆 Best model: LightGBM (Tuned) (R²=0.8825)


In [23]:
feature_order = X_train.columns.tolist()

In [25]:
feature_order

['State',
 'Crop',
 'Season',
 'Annual_Rainfall',
 'Fertilizer',
 'Pesticide',
 'Rainfall_per_Fertilizer',
 'Fertilizer_Pesticide_Ratio',
 'Total_Input',
 'Rainfall_Squared',
 'Fertilizer_Squared',
 'Log_Rainfall',
 'Log_Fertilizer']

In [29]:
joblib.dump(best_lgb, '../backend/ml/best_model_enhance.pkl')
joblib.dump(encoders, '../backend/ml/encoders_enhance.pkl')
joblib.dump(feature_order, '../backend/ml/feature_order_enhance.pkl')
print(f"\n✅ Saved LightGBM model")
print(f"   Features: {len(feature_order)}")
print(f"   R²: {lgb_r2:.4f}")
print(f"   MAE: {lgb_mae:.4f}")


✅ Saved LightGBM model
   Features: 13
   R²: 0.8825
   MAE: 23.4781


In [31]:
print("\n" + "="*60)
print("TESTING ON YOUR EXAMPLE")
print("="*60)
# Your test case
test_case = pd.DataFrame([{
    'Crop': 'arhar/tur',
    'State': 'andhra pradesh',
    'Season': 'kharif',
    'Annual_Rainfall': 928.4,
    'Fertilizer': 102.13,
    'Pesticide': 0.26
}])
# Apply same feature engineering
test_case['Rainfall_per_Fertilizer'] = test_case['Annual_Rainfall'] / (test_case['Fertilizer'] + 1)
test_case['Fertilizer_Pesticide_Ratio'] = test_case['Fertilizer'] / (test_case['Pesticide'] + 0.01)
test_case['Total_Input'] = test_case['Fertilizer'] + test_case['Pesticide']
test_case['Rainfall_Squared'] = test_case['Annual_Rainfall'] ** 2
test_case['Fertilizer_Squared'] = test_case['Fertilizer'] ** 2
test_case['Log_Rainfall'] = np.log1p(test_case['Annual_Rainfall'])
test_case['Log_Fertilizer'] = np.log1p(test_case['Fertilizer'])
# Encode
for col in categorical_cols:
    if col in test_case.columns:
        test_case[col] = encoders[col].transform(test_case[col])
        test_case[col] = test_case[col].astype('category')
# Reorder
test_case = test_case[feature_order]
# Predict
pred = best_lgb.predict(test_case)[0]
actual = 0.468989
print(f"\nYour test case:")
print(f"  Predicted: {pred:.6f}")
print(f"  Actual:    {actual:.6f}")
print(f"  Error:     {abs(pred - actual):.6f} ({abs(pred - actual)/actual*100:.2f}%)")
print("\n" + "="*60)
print("✅ TRAINING COMPLETE!")
print("="*60)
print(f"\nNext step: Update backend/utils/preprocess.py to include feature engineering")


TESTING ON YOUR EXAMPLE

Your test case:
  Predicted: 1.136249
  Actual:    0.468989
  Error:     0.667260 (142.28%)

✅ TRAINING COMPLETE!

Next step: Update backend/utils/preprocess.py to include feature engineering
